In [0]:
%run "./Data_Transform_Factory"

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

class Transform():
    """
    Most sold pizza in Chicken Category(Large, Medium, Small) month wise

    """
    def requirementOne(self,inputs):
        df = Derivate.derivate(self,inputs)

        windowSpec = Window\
            .partitionBy("Order_Month","name")\
            .orderBy(desc("total_sales"),"size")

        scenarioOneDf =  df\
            .groupBy("name","size","Order_Month","category")\
            .agg(count("order_id").alias("total_sales"))\
            .withColumn("Rank",rank().over(windowSpec))\
            .filter((col("category")=="Chicken") & (col("Rank") == 1))\
            .selectExpr("name as Pizza_Name","size as Pizza_Size","total_sales","Order_Month")

        print("Most sold pizzas in every chicken pizzas month wise")
        scenarioOneDf.show()
        return scenarioOneDf
    
    """
    Most sold pizza in every Category overall
    """
    def requirementTwo(self,inputs):
        df = Derivate.derivate(self,inputs)

        windowSpec = Window\
            .partitionBy("category")\
            .orderBy(desc("total_sales"))

        scenarioTwoDf = df\
            .groupBy("category","size","name")\
            .agg(count("order_id").alias("Total_Sales"))\
            .withColumn("Rank",rank().over(windowSpec))\
            .selectExpr("name as Pizza_Name","size as Pizza_Size","Total_Sales","Category")\
            .filter(col("Rank") == 1)

        print("Most sold pizza in every Category overall")
        scenarioTwoDf.show()
        return scenarioTwoDf
    
        """
    Most sold pizza during 7.00pm to 10.00pm overall hour wise
    """
    def requirementThree(self,inputs):
        df = Derivate.derivate(self,inputs)

        windowSpec = Window\
            .partitionBy("Order_Hour")\
            .orderBy(desc("Total_Sales"))

        finalDf = df\
            .groupBy("Order_Hour","name","size")\
            .agg(count("order_id").alias("Total_Sales"))\
            .withColumn("Rank",rank().over(windowSpec))\
            .selectExpr("name as Pizza_Name","size as Size","Total_Sales","Order_Hour")\
            .filter((col("Rank")==1) & (col("Order_Hour")>16) & (col("Order_Hour")<23))
        print("Most sold pizza during 7.00pm to 10.00pm overall hour wise")
        finalDf.show()
        return finalDf

        